<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Objectives" data-toc-modified-id="Objectives-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Objectives</a></span></li><li><span><a href="#Example-Together" data-toc-modified-id="Example-Together-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Example Together</a></span><ul class="toc-item"><li><span><a href="#Question" data-toc-modified-id="Question-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Question</a></span></li><li><span><a href="#Considerations" data-toc-modified-id="Considerations-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Considerations</a></span></li><li><span><a href="#Loading-the-Data" data-toc-modified-id="Loading-the-Data-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Loading the Data</a></span></li><li><span><a href="#Some-Exploration-to-Better-Understand-our-Data" data-toc-modified-id="Some-Exploration-to-Better-Understand-our-Data-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Some Exploration to Better Understand our Data</a></span></li><li><span><a href="#Experimental-Setup" data-toc-modified-id="Experimental-Setup-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Experimental Setup</a></span><ul class="toc-item"><li><span><a href="#What-Test-Would-Make-Sense?" data-toc-modified-id="What-Test-Would-Make-Sense?-2.5.1"><span class="toc-item-num">2.5.1&nbsp;&nbsp;</span>What Test Would Make Sense?</a></span></li><li><span><a href="#The-Hypotheses" data-toc-modified-id="The-Hypotheses-2.5.2"><span class="toc-item-num">2.5.2&nbsp;&nbsp;</span>The Hypotheses</a></span></li><li><span><a href="#Setting-a-Threshold" data-toc-modified-id="Setting-a-Threshold-2.5.3"><span class="toc-item-num">2.5.3&nbsp;&nbsp;</span>Setting a Threshold</a></span></li></ul></li><li><span><a href="#$\chi^2$-Test" data-toc-modified-id="$\chi^2$-Test-2.6"><span class="toc-item-num">2.6&nbsp;&nbsp;</span>$\chi^2$ Test</a></span><ul class="toc-item"><li><span><a href="#Setup-the-Data" data-toc-modified-id="Setup-the-Data-2.6.1"><span class="toc-item-num">2.6.1&nbsp;&nbsp;</span>Setup the Data</a></span></li><li><span><a href="#Calculation" data-toc-modified-id="Calculation-2.6.2"><span class="toc-item-num">2.6.2&nbsp;&nbsp;</span>Calculation</a></span></li></ul></li><li><span><a href="#Interpretation" data-toc-modified-id="Interpretation-2.7"><span class="toc-item-num">2.7&nbsp;&nbsp;</span>Interpretation</a></span></li></ul></li><li><span><a href="#Exercise" data-toc-modified-id="Exercise-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Exercise</a></span></li></ul></div>

In [2]:
import numpy as np
import pandas as pd

from scipy import stats
import seaborn as sns

# Objectives

- Conduct an A/B test in Python
- Interpret the results of the A/B tests for a stakeholder

# Example Together

## Question

We have data about whether customers completed sales transactions, segregated by the type of ad banners to which the customers were exposed.

The question we want to answer is whether there was any difference in sales "conversions" between desktop customers who saw the sneakers banner and desktop customers who saw the accessories banner in the month of May 2019.

## Considerations

What would we need to consider when designing our experiment?

Might include:

- Who is it that we're including in our test?
- How big of an effect would make it "worth" us seeing?
    - This can affect sample size
    - This can give context of a statistically significant result
- Other biases or "gotchas"

## Loading the Data

First let's download the data from [kaggle](https://www.kaggle.com/podsyp/how-to-do-product-analytics) via the release page of this repo: https://github.com/flatiron-school/ds-ab_testing/releases 

The code below will load it into our DataFrame:

In [3]:
# This will download the data from online so it can take
# some time (but relatively small download)

df = pd.read_csv('https://github.com/flatiron-school/ds-ab_testing/\
releases/download/v1.2/products_small.csv')

> Let's take a look while we're at it

In [4]:
# Check it out
df.head()

,Unnamed: 0,order_id,user_id,page_id,product,site_version,time,title,target
0,4122928,3e6c5e89fdddcaee0eed210ec2c9cadf,90d58d967eb72656e86059ec6f208092,2fdc16a09e0016555dd4da4a3fe84414,accessories,desktop,2019-03-06 08:42:47,banner_show,0
1,564306,feed6203517d3abf6aab13761633174b,08703dab1f004eabba25aacb7f0e5484,6b0a902b9b73d5a158d0119d6feb38ac,sneakers,mobile,2019-04-19 18:30:45,banner_show,0
2,1872289,e33d5d7941edc281646aa37763729771,bdf1d25697e21419901c94fabdafad15,9ddb7315c4357929931b48f2b3d11c62,company,mobile,2019-01-20 17:20:10,banner_show,0
3,3616779,7c4caa8d508fa7c3bbc25f35cdd9168a,8d2f23a732c9527d95678088a3bac122,1f86cd0bea31d54a5b511b42fd19401a,sneakers,mobile,2019-02-20 09:38:32,banner_show,0
4,5871482,12874b29bde8bbd43fb2b95735caf9e6,5a22604f8f31ae98ee1211ece3a02004,b533f7e2003418c63fd71471264c559a,sneakers,mobile,2019-04-24 09:19:02,banner_show,0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 9 columns):
 #   Column        Non-Null Count    Dtype 
---  ------        --------------    ----- 
 0   Unnamed: 0    1000000 non-null  int64 
 1   order_id      1000000 non-null  object
 2   user_id       1000000 non-null  object
 3   page_id       1000000 non-null  object
 4   product       1000000 non-null  object
 5   site_version  1000000 non-null  object
 6   time          1000000 non-null  object
 7   title         1000000 non-null  object
 8   target        1000000 non-null  int64 
dtypes: int64(2), object(7)
memory usage: 68.7+ MB


In [6]:
df.describe()

,Unnamed: 0,target
count,1.000000e+06,1000000.000000
mean,4.239850e+06,0.029395
std,2.443943e+06,0.168911
min,4.000000e+00,0.000000
25%,2.125160e+06,0.000000
50%,4.244678e+06,0.000000
75%,6.355990e+06,0.000000
max,8.471211e+06,1.000000


In [7]:
df.describe(include=['object'])

,order_id,user_id,page_id,product,site_version,time,title
count,1000000,1000000,1000000,1000000,1000000,1000000,1000000
unique,1000000,882589,959023,5,2,952092,3
top,80428d02e5d3ed8469b7f579efe2fc0e,2a98f631f947487fc710b89d13d5a51a,a3d2de7675556553a5f08e4c88d2c228,clothes,mobile,2019-01-14 18:47:47,banner_show
freq,1,8,29395,210996,718521,5,872275


In [ ]:
# All the deets


## Some Exploration to Better Understand our Data

Lets's look at the different banner types:

In [10]:
# banner types

df['product'].value_counts()

clothes             210996
company             203020
sneakers            201298
sports_nutrition    193200
accessories         191486
Name: product, dtype: int64

In [11]:
df.site_version.value_counts()

mobile     718521
desktop    281479
Name: site_version, dtype: int64

In [14]:
# products and targets
df.groupby('product')['target'].value_counts(normalize=True)

product           target
accessories       0         0.972014
                  1         0.027986
clothes           0         0.937079
                  1         0.062921
company           0         1.000000
sneakers          0         0.960640
                  1         0.039360
sports_nutrition  0         0.985316
                  1         0.014684
Name: target, dtype: float64

Let's look at the range of time-stamps on these data:

In [15]:
# Date range
df['time'].min()

'2019-01-01 00:00:25'

In [16]:
df['time'].max()

'2019-05-31 23:59:21'

Let's check the counts of the different site_version values:

In [ ]:
# Site types


In [17]:
# action title
df['title'].value_counts()

banner_show     872275
banner_click     98330
order            29395
Name: title, dtype: int64

In [18]:
# double check
df['target'].value_counts()

0    970605
1     29395
Name: target, dtype: int64

In [19]:
sum(df['target'])

29395

In [22]:
len(df['title'].loc[ df['title'] == 'order'])

29395

In [24]:
df.groupby('title').count()

,Unnamed: 0,order_id,user_id,page_id,product,site_version,time,target
title,,,,,,,,
banner_click,98330,98330,98330,98330,98330,98330,98330,98330
banner_show,872275,872275,872275,872275,872275,872275,872275,872275
order,29395,29395,29395,29395,29395,29395,29395,29395


In [27]:
o_f = df['title'] == 'order'

t_f  = df['target'] == 1

In [29]:
df.loc[ (o_f) & (t_f)  ].shape

(29395, 9)

In [30]:
df['target2'] = df['title'].apply(lambda x: 1 if x == 'order' else 0)

In [31]:
df.head()

,Unnamed: 0,order_id,user_id,page_id,product,site_version,time,title,target,target2
0,4122928,3e6c5e89fdddcaee0eed210ec2c9cadf,90d58d967eb72656e86059ec6f208092,2fdc16a09e0016555dd4da4a3fe84414,accessories,desktop,2019-03-06 08:42:47,banner_show,0,0
1,564306,feed6203517d3abf6aab13761633174b,08703dab1f004eabba25aacb7f0e5484,6b0a902b9b73d5a158d0119d6feb38ac,sneakers,mobile,2019-04-19 18:30:45,banner_show,0,0
2,1872289,e33d5d7941edc281646aa37763729771,bdf1d25697e21419901c94fabdafad15,9ddb7315c4357929931b48f2b3d11c62,company,mobile,2019-01-20 17:20:10,banner_show,0,0
3,3616779,7c4caa8d508fa7c3bbc25f35cdd9168a,8d2f23a732c9527d95678088a3bac122,1f86cd0bea31d54a5b511b42fd19401a,sneakers,mobile,2019-02-20 09:38:32,banner_show,0,0
4,5871482,12874b29bde8bbd43fb2b95735caf9e6,5a22604f8f31ae98ee1211ece3a02004,b533f7e2003418c63fd71471264c559a,sneakers,mobile,2019-04-24 09:19:02,banner_show,0,0


## Experimental Setup

We need to filter by site_version, time, and product:

In [ ]:
site_version = desktop
time = May
product = sneakers and accessories

In [33]:
'a' > 'b'

False

In [42]:
'2019-05-01' < '2019-05-01 00:00:00 '

True

In [44]:
desk_filter = df['site_version'] == 'desktop'

time_filter = df['time'] > '2019-05-01'

product_filter = df['product'].isin(['sneakers','accessories'])



In [45]:
df2 = df.loc[desk_filter & time_filter & product_filter].copy()

In [47]:
df2.head()

,Unnamed: 0,order_id,user_id,page_id,product,site_version,time,title,target,target2
32,8064199,f909eb9bbf2795337dd38323b7951705,5a96585c3db94eb2ae40e559cbd2e164,94b6ea2316b773afa20925d9b79d5bf8,accessories,desktop,2019-05-19 09:26:29,banner_show,0,0
38,7560076,97808cfd3d44fa7e8cfeb71bb5f02590,1e2db5d01051fc65fffc6c27f34400a3,48c8e3f287d814493cff887ff445f535,sneakers,desktop,2019-05-03 05:34:46,banner_show,0,0
59,5923209,71c39cc2e2ede95a7ca296d4e3e56f85,46a6f13750d5032f366349e3e3171678,30559d0b23980e7a778c93c52ba5deca,sneakers,desktop,2019-05-19 05:04:32,banner_show,0,0
105,8161379,e969a575bcf5b10d6676c2fab9cbf784,aa77bca62e9a84f5cd8d734c70afd060,70ed6e66458fda7a3164e2633d81ea07,sneakers,desktop,2019-05-29 17:51:46,banner_show,0,0
115,4564531,3068b616fac2838de6a2d33a6fad60ad,aec8ee5450418b5bc97127c2269058ca,14cbe8cf92c259f5d18310347e3cb5cb,sneakers,desktop,2019-05-04 09:33:44,banner_show,0,0


In [51]:
df2.site_version.value_counts()


desktop    25972
Name: site_version, dtype: int64

In [53]:
df2['product'].value_counts()

sneakers       13269
accessories    12703
Name: product, dtype: int64

In [ ]:
# sniff test!

### What Test Would Make Sense?

Since we're comparing the frequency of conversions of customers who saw the "sneakers" banner against those who saw the "accessories" banner, we can use a $\chi^2$ test.

Note there are other hypothesis tests we can use but this should be fine since it should fit our criteria.

### The Hypotheses

$H_0$: There is no difference between the two

$H_1$: There is a difference

### Setting a Threshold

We'll set a false-positive rate of $\alpha = 0.05$.

## $\chi^2$ Test

### Setup the Data

We need our contingency table: the numbers of people who did or did not submit orders, both for the accessories banner and the sneakers banner. 

In [54]:
# Create the two product types
df_A = df2.loc[ df2['product'] == 'accessories']
df_B = df2.loc[ df2['product'] == 'sneakers']

In [55]:
# Get number of orders
accessories_orders = sum(df_A['target'])
sneakers_orders = sum(df_B['target'])

In [56]:
accessories_orders, sneakers_orders

(496, 799)

To get the numbers of people who didn't submit orders, we get the total number of people who were shown banners and then subtract the numbers of people who did make orders.

In [ ]:
len, shape, 

In [66]:
# get conversion rates

accessories_total = len(df_A)
sneakers_total = len(df_B)


accessories_no_order = accessories_total - accessories_orders
sneakers_no_order = sneakers_total - sneakers_orders

In [67]:
accessories_no_order, sneakers_no_order

(12207, 12470)

In [68]:
df_A.shape

(12703, 10)

In [69]:
accessories_no_order + accessories_orders

12703

In [74]:
# Create table
contingency_table = np.array([
    [accessories_orders, accessories_no_order],
    [sneakers_orders, sneakers_no_order]
])

In [75]:
contingency_table

array([[  496, 12207],
       [  799, 12470]])

### Calculation


In [ ]:
stats.chisquare()

In [76]:
# Chi^2!

stats.chi2_contingency(contingency_table)

(60.94640624666488,
 5.86498985882812e-15,
 1,
 array([[  633.38922686, 12069.61077314],
        [  661.61077314, 12607.38922686]]))

This extremely low $p$-value suggests that these two groups are genuinely performing differently. In particular, the desktop customers who saw the sneakers banner in May 2019 bought at a higher rate than the desktop customers who saw the accessories banner in May 2019.

## Interpretation

In [77]:
contingency_table

array([[  496, 12207],
       [  799, 12470]])

In [80]:
contingency_table[:,0]

array([496, 799])

In [82]:
len(df_A)

12703

In [81]:
contingency_table.sum(axis=1)

array([12703, 13269])

In [78]:
# Find the difference in conversion rate
accessory_CR, sneaker_CR = contingency_table[:, 0] / contingency_table.sum(axis=1)

In [79]:
print(f'Conversion Rate for accessory banner:\n\t{100*accessory_CR:.3f}%')
print(f'Conversion Rate for sneaker banner:\n\t{100*sneaker_CR:.3f}%')
print('')
print(f'Absolute difference of CR: {100*(sneaker_CR-accessory_CR):.3f}%')

Conversion Rate for accessory banner:
	3.905%
Conversion Rate for sneaker banner:
	6.022%

Absolute difference of CR: 2.117%



So we can say:
- There was a statistically significant difference at the $\alpha$-level (confidence level)
- The difference was about $2.1\%$ in favor of the sneaker banner!

# Exercise

> The company is impressed with what you found and is now wondering if there is a difference in their other banner ads!

With your group, look at the same month (May 2019) but compare different platforms ('mobile' vs 'desktop') and or different banner types ('accessories', 'sneakers', 'clothes', 'sports_nutrition'). Just don't repeat the same test we did above 😉

Make sure you record what considerations you have for the experiment, what hypothesis test you performed ($H_0$ and $H_1$ too), and your overall conclusion/interpretation for the _business stakeholders_. Is there a follow-up you'd suggest? 

H0 = no difference in conversion rate
HA = there is a difference

In [84]:
# Charlie

site_f2 = df['site_version'] == 'mobile'
time_f2 = df['time'] > '2019-05-01'
product_f2 = df['product'].isin(['clothes', 'accessories'])

df3 = df.loc[site_f2 & time_f2 & product_f2].copy()



df_a = df3.loc[df3['product'] == 'accessories']
df_b = df3.loc[df3['product'] == 'clothes']


accessories_orders = sum(df_a['target'])
clothes_orders = sum(df_b['target'])

accessories_total = len(df_a)
clothes_total = len(df_b)

accessories_no_order = accessories_total - accessories_orders
clothes_no_order = clothes_total - clothes_orders
clothes_no_order, accessories_no_order


contingency_table = np.array([
    (accessories_orders, accessories_no_order),
    (clothes_orders, clothes_no_order)
])

# contingency_table

stats.chi2_contingency(contingency_table)

# Find the difference in conversion rate
accessory_CR, clothes_CR = contingency_table[:, 0] / contingency_table.sum(axis=1)

print('Mobile Results')
print(f'Conversion Rate for accessory banner:\n\t{100*accessory_CR:.3f}%')
print(f'Conversion Rate for sneaker banner:\n\t{100*clothes_CR:.3f}%')
print('')
print(f'Absolute difference of CR: {100*(clothes_CR-accessory_CR):.3f}%')

accessory_CR, clothes_CR

print(''' Desktop Results:

Conversion Rate for accessory banner:
	3.905%
Conversion Rate for sneaker banner:
	10.718%

P-Value: 
     7.124173664285203e-101

Absolute difference of CR: 6.813%''')

Mobile Results
Conversion Rate for accessory banner:
	1.734%
Conversion Rate for sneaker banner:
	2.833%

Absolute difference of CR: 1.098%
 Desktop Results:

Conversion Rate for accessory banner:
	3.905%
Conversion Rate for sneaker banner:
	10.718%

P-Value: 
     7.124173664285203e-101

Absolute difference of CR: 6.813%


In [85]:
# Ian

mobile_filter = df['site_version'] == 'mobile'

df3 = df.loc[mobile_filter & time_filter & product_filter].copy()

df3['product'].value_counts()

df_A_m = df3[df3['product'] == 'accessories']
df_B_m = df3[df3['product'] == 'sneakers']

accessories_orders_m = sum(df_A_m['target'])
sneakers_orders_m = sum(df_B_m['target'])

accessories_total_m = len(df_A_m)
sneakers_total_m = len(df_B_m)

accessories_no_order_m = accessories_total_m - accessories_orders_m
sneakers_no_order_m = sneakers_total_m - sneakers_orders_m

accessories_no_order_m + accessories_orders_m

contingency_table_m = np.array([
    [accessories_orders_m, accessories_no_order_m],
    [sneakers_orders_m, sneakers_no_order_m]
])

contingency_table_m

accessory_CR_m, sneaker_CR_m = contingency_table_m[:, 0] / contingency_table_m.sum(axis=1)

print(f'Conversion Rate for accessory banner:\n\t{100*accessory_CR_m:.3f}%')
print(f'Conversion Rate for sneaker banner:\n\t{100*sneaker_CR_m:.3f}%')
print('')
print(f'Absolute difference of CR: {100*(sneaker_CR_m-accessory_CR_m):.3f}%')

Conversion Rate for accessory banner:
	1.734%
Conversion Rate for sneaker banner:
	2.463%

Absolute difference of CR: 0.729%


In [87]:
# Mitch, Garret, and Charles
df['site_version'].value_counts()

time_filter = df['time'] >= '2019-05-01'

df3 = df.loc[time_filter]

df3.head()

# Null: There is no difference in conversion rate between mobile and desktop versions.

# Alt: There is a difference between mobile and desktop conversion rates.

# Create new dataframes for each site_versions
mobile_filter = df3['site_version'] == 'mobile'
desktop_filter = df3['site_version'] == 'desktop'

# Create new dataframes for each site_versions
df_m = df3.loc[mobile_filter]
df_d = df3.loc[desktop_filter]

# finding total people who bought and didn't buy
# mobile 
mobile_orders = sum(df_m['target'])
mobile_no_orders = len(df_m) - mobile_orders

# desktop
desktop_orders = sum(df_d['target'])
desktop_no_orders = len(df_d) - desktop_orders

print(mobile_orders, mobile_no_orders)
print(desktop_orders, desktop_no_orders)

len(df_m), len(df_d)

# Make a contingency table
contingency_table = np.array([
    [mobile_orders, mobile_no_orders],
    [desktop_orders, desktop_no_orders]    
])
contingency_table

print(stats.chi2_contingency(contingency_table))

# Find the difference in conversion rate
mobile_CR, desktop_CR = contingency_table[:, 0] / contingency_table.sum(axis=1)

print(f'Conversion Rate for mobile banner:\n\t{100*mobile_CR:.3f}%')
print(f'Conversion Rate for desktop banner:\n\t{100*desktop_CR:.3f}%')
print('')
print(f'Absolute difference of CR: {100*(desktop_CR-mobile_CR):.3f}%')

2939 180996
3224 64616
(2063.9893282896182, 0.0, 1, array([[  4502.39859001, 179432.60140999],
       [  1660.60140999,  66179.39859001]]))
Conversion Rate for mobile banner:
	1.598%
Conversion Rate for desktop banner:
	4.752%

Absolute difference of CR: 3.155%


In [88]:
results = stats.chi2_contingency(contingency_table)

In [89]:

results

(2063.9893282896182,
 0.0,
 1,
 array([[  4502.39859001, 179432.60140999],
        [  1660.60140999,  66179.39859001]]))

In [90]:
a,p,dof,vals = results

In [91]:
p

0.0